In [1]:
####################################
#LIBRARIES

import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as dde
import matplotlib.pyplot as plt
import json

####################################
#Parameters
samplenum = 50
input_size = 3
output_size = 3
nTimeSteps = 10; #at 60 Hz
model_file_path = '../Trained_Models/'
data_file_path = '../Data/Samples/data_20k_2x2x2/'
simulation_file_path = '../Data/Simulations/pointmass.sim'
objective_file_path = '../Data/Objectives/pointmass.opt'

#######################################
# LOAD SIMULATION AND OBJECTIVE FUNCTION
dyn = dde.DynamicSequence()
dyn.loadFile(simulation_file_path, nTimeSteps)
p_init = np.zeros(dyn.p0.size*nTimeSteps)
for i in range(nTimeSteps):
	p_init[i*dyn.p0.size : (i+1)*dyn.p0.size] = dyn.p0

#############################################
#LOAD OBJECTIVE PYDDE_V2
obj = dde.InverseObjective(dyn)
obj.loadFile(objective_file_path)
objective_json = json.load(open(objective_file_path))

#############################################
#GENERATE OPTIMIZATION PYDDE_V2
opt = dde.Newton()

#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
print('cool2')


cool2


In [ ]:
###################################
#USE P FROM TRAJ_OPT
p = np.ones((3*nTimeSteps, samplenum))
for i in range(samplenum):
    objective_json["objectives"]["pmTargetPositions"][0]["targetPos"] = ([[y_target[i,0]],[y_target[i,1]],[y_target[i,2]]]) 
    obj.loadJson(objective_json)
    p[:,i] = opt.minimize(obj, p_init)
print('hello')
p = torch.tensor(p, requires_grad = True).t()
print(p)

In [1]:
import sys
print(sys.path)

['/Users/nicovonau/Code/thesis_pytorch/Examples', '/private/var/folders/fg/z9hxlwkx7dl424b9sm5btqmh0000gn/T/cbe117fc-46b2-4cfe-b7bd-e90c47981a88', '/Users/nicovonau/.vscode/extensions/ms-python.python-2020.6.91350/pythonFiles', '/Users/nicovonau/.vscode/extensions/ms-python.python-2020.6.91350/pythonFiles/lib/python', '/Users/nicovonau/anaconda3/lib/python37.zip', '/Users/nicovonau/anaconda3/lib/python3.7', '/Users/nicovonau/anaconda3/lib/python3.7/lib-dynload', '', '/Users/nicovonau/anaconda3/lib/python3.7/site-packages', '/Users/nicovonau/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/nicovonau/pathpy', '/Users/nicovonau/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/nicovonau/.ipython']


In [7]:
# Generate simulation, random p
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
#print(y_target)
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
p = torch.tensor(p, requires_grad = True).t()
input = p.double()
print(input.shape)

torch.Size([2, 9])


In [13]:
################################
#BUILD CUSTOM FUNCTION
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        print(f'y_pred: {y_pred}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.dtype}')
        
        grad_input = torch.tensor(grad_output.double().mm(dy_dp_batch/samplenum))
        print(f'shape of grad input: {grad_input.shape}')
        print(f'shape of grad output: {grad_output.shape}')
        print(f'grad_input: {grad_input}')
        print(f'grad_output: {grad_output}')
        return grad_input    

Simulate = Simulate.apply

In [17]:
#GRADCHECK
from torch.autograd import gradcheck

test = gradcheck(Simulate, (input,), eps=2e-3, atol=1e-4, raise_exception = True)
print(test)

y_pred: tensor([[2.1556e-05, 1.9997e+00, 0.0000e+00]])
shape of dy/dp_batch: torch.float64
shape of grad input: torch.Size([1, 9])
shape of grad output: torch.Size([1, 3])
grad_input: tensor([[ 8.0987e-03,  1.4527e-06,  0.0000e+00,  5.4188e-03, -3.7828e-06,
          0.0000e+00,  2.7168e-03, -1.9345e-05,  0.0000e+00]],
       dtype=torch.float64)
grad_output: tensor([[1., 0., 0.]])
shape of dy/dp_batch: torch.float64
shape of grad input: torch.Size([1, 9])
shape of grad output: torch.Size([1, 3])
grad_input: tensor([[ 8.0987e-03,  1.4527e-06,  0.0000e+00,  5.4188e-03, -3.7828e-06,
          0.0000e+00,  2.7168e-03, -1.9345e-05,  0.0000e+00]],
       dtype=torch.float64)
grad_output: tensor([[1., 0., 0.]])
shape of dy/dp_batch: torch.float64
shape of grad input: torch.Size([1, 9])
shape of grad output: torch.Size([1, 3])
grad_input: tensor([[ 1.3361e-06, -5.3950e-02,  0.0000e+00, -3.7514e-06,  1.7934e-01,
          0.0000e+00, -1.9345e-05,  9.0041e-01,  0.0000e+00]],
       dtype=torch.